Unsupervised Merge DTW Matrix</br>
Dependencies: EIA_Plant_Data.ipynb (Plant_Location_{state}.csv files for all states to use in analysis)</br>
Output: Merge_Norm.csv, Merge_Raw.csv, DTW_Matrix_Raw.csv, DTW_Matrix_Norm.csv (as well as No0s versions of DTWs if desired)</br></br>
Given a list of state codes matching csv files of gas consumption data, this notebook creates time series representations of both the raw and normalized gas consumption for each plant in these states and saves them as csvs. Using these time series, this notebook then creates Dynamic Time Warping distance matrices for both the raw data consumption time series and the normalized data consumption time series for all the plants in these states.

In [5]:
import numpy as np
import pandas as pd

# Suppress all warnings
import warnings
warnings.filterwarnings("ignore")
import math

/var/folders/lq/0q3fzmb57s3c187k6gjyx3_00000gn/T/ipykernel_27994/852975974.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [63]:
continental_states = ['AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
           'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

In [3]:
states = ['MI', 'WA','TX', 'FL']

In [65]:
#Cutoff date - otherwise will go to 2023-12-31
date_cutoff = '2022-12-31'

In [120]:
def cleaned_state (state, normalize=None):
    '''Input: 2-letter state abbreviation
    Ouput: dataframe for the state that cleaned and columns paired down, and normalized if desired'''
    df = pd.read_csv(f'../Cleaning/Plant_Location_{state}.csv')
    df['period'] = pd.to_datetime(df['period'], format='%Y-%m').dt.to_period(freq="D")
    df= df.dropna(axis=0)
    
    df.rename(columns={"total-consumption": "consumption"}, inplace=True)
    
    df = df[['state','plantCode', 'period','consumption', 'Longitude', 'Latitude']]
    
    #issue with FL - some consumption numbers are "Natural Gas = 815.8 MW" (found with try: int(x), except: print)
    #All non-numbers filled in with 0s...
    df['consumption'] = pd.to_numeric(df['consumption'], errors='coerce')

    df[['Longitude','Latitude','consumption']] = df[['Longitude','Latitude','consumption']].fillna(0)
    df['plantCode'] = df['plantCode'].astype(np.int64)
    
    if normalize:
        df['consumption']= df['consumption'] / df[['consumption','plantCode']].groupby('plantCode').transform('sum')['consumption']
    
    return df

In [27]:
def multi_state_DF (states, normalize=None):
    '''Input: list of state abbreviations
    Output: Merged dataframe of all the states: with time periods as index and ST_plantcodes as columns'''
    for state in states:
        temp = cleaned_state(state, normalize)
        try:
            df = pd.concat([df,temp], axis=0)
        except:
            df = temp
    
    return df

In [80]:
def pivoted_df (df, dezero=None):
    '''Input df with data for all the states, and whether time series with 0 values should be dropped
    Ouput: dataframe for the state that is formated with time periods as index and plantCodes as columns'''
    df = df[['plantCode', 'period','consumption']]

    df = pd.pivot_table(df, values='consumption', index=['period'], columns=['plantCode'])
    
    #get rid of columns (plants) that have 0 values?
    if dezero:
        df = df.replace(0.0,np.nan)
        df = df.dropna(axis=1)

    df = df.fillna(0)

    #Drop data for 2023!
    for row in df.index:
        if row >= pd.to_datetime(date_cutoff).to_period(freq="D"):
            df.drop(row, axis=0, inplace=True)
    
    return df

In [67]:
def calc_pairwise_dtw_cost(x, y, ret_matrix=False):
    """
    Takes in two series. If ret_matrix=True, returns the full DTW cost matrix;
    otherwise, returns only the overall DTW cost
    """
    cost_matrix = np.zeros((len(x), len(y)))
    dtw_cost = None
    dist_fn = lambda a, b: (a - b) ** 2  # Optional helper function
    
    for i in range(len(x)): #x = i = row
        for j in range(len(y)): # y = j = col
        #If the coordinate is (0,0): define the cost matrix as d(x_(i),y_(j))
            if i==0 and j==0:
                cost_matrix[i,j]=dist_fn(x[i],y[j])
        #Otherwise;
            else:
            #the cost matrix is equal to the sum of d(x_(i),y_(j)) and the minimum of the following:
    #                         1)  If the row is less than or equal to zero;
                if (i-1) < 0:
                    up = math.inf
                else:
                    up = cost_matrix[j, i -1]
    #                                      * compute the cost matrix using i and j-1
    #                              Else: too large to compute
    #                          2)  If the column is less than or equal to zero;
                if (j-1) < 0:
                    left = math.inf
                else:
                    left = cost_matrix[j - 1, i]
    #                            * compute the cost matrix using i-1 and j
    #                              Else: too large to compute
    #                          3)  If the row and columns are less than or equal to zero;
                if (j-1) >= 0 and (i-1) >= 0:
                    diag =  cost_matrix[j-1, i-1] # i ==0, j==0 case already handled
        #                                    * too large to compute
                else:
                    diag = math.inf
                cost = min(up,left,diag)

                cost_matrix[j,i] = dist_fn(x[i],y[j]) + cost
    dtw_cost = cost_matrix[-1][-1]
    
    return cost_matrix if ret_matrix else dtw_cost

In [12]:
def calc_dtw_cost(df):
    """
    Takes in a DataFrame and computes all pairwise DTW costs
    """
    
    dtw_cost_df = pd.DataFrame(np.zeros((len(df.columns),len(df.columns))),
                              index=df.columns,
                              columns=df.columns)
    
    for i in range(len(df.columns)):
        for j in range(len(df.columns)):
            if i == j:
                pass
            else:
                dtw_cost_df.iloc[i,j] = calc_pairwise_dtw_cost(df.iloc[:,i], df.iloc[:,j])
    
    return dtw_cost_df

In [13]:
def DTW_distance_matrix (df, output='DTW_Matrix.csv'):
    '''Inupt: list of state abbreviations, tag to normalize or not
    Output: Dynamic Time Warp distance matrix for all plants in states list'''

    DTW = calc_dtw_cost(df)
    
    #Handy if computing large matrix, time intensize to run
    DTW.to_csv(output, index=False)
    
    return DTW    

In [115]:
def merge_and_DTW (states, normalize=None, output=None):
    '''Input: state - a list of states
        normalize - if True, then the consumption values for each plant will be normalized
        dezero - if True, then removes all plants that have a value of 0 consumption at some point
        output - the file name signifier for the output file
    Output: saves specified DTW distance matrix and merged datafile'''
    
    raw_df = multi_state_DF (states, normalize=normalize)
    
    df = pivoted_df(raw_df, dezero=None)
    DTW_distance_matrix (df, output=f'DTW_Matrix_{output}.csv')
    
    timeseries = df.T
    timeseries.reset_index(inplace=True)
    dict_df = raw_df[['state', 'plantCode','Longitude', 'Latitude']].drop_duplicates()
    merge = pd.merge(dict_df, timeseries, how='left', left_on='plantCode', right_on='plantCode')
    merge = merge.fillna(0.0)

    merge.to_csv(f'Merge_{output}.csv', index=False)
    
    #df = pivoted_df(raw_df, dezero=True)
    #DTW_distance_matrix (df, output=f'DTW_Matrix_{output}_No0.csv')
    
    return print(f'Success for {output}')
        

In [31]:
def create_merge_DTWs (states):
    merge_and_DTW(states, output='Raw')
    merge_and_DTW(states, normalize=True, output='Norm')
    
    return print(f'Total success for {states}')

In [121]:
create_merge_DTWs (states)

Success for Raw
Success for Norm
Total success for ['MI', 'WA', 'TX', 'FL']


In [122]:
df = pd.read_csv('Merge_Raw.csv')
len(df), len(df.columns)

(503, 52)

In [11]:
def find_missing_data (states):
    '''Input: list of states used in the data anaylsis
    Output: Prints out statistics on data and how many missing values were imputed with 0s when creating times serieses'''

    data = 0
    for state in states:
        df  = pd.read_csv(f'../Cleaning/Plant_Location_{state}.csv')
        data += len(df)
    print(f'{data} NG Gas Consumption Data Points')

    time_series = pd.read_csv(f'Merge_Raw.csv')
    periods = len(time_series.columns)-4
    series_data = len(time_series) * periods

    missing = series_data - data

    print(f'{series_data} Data Points in Time Series Data')
    print(f'{missing} Missing Data Points Imputed with 0s')

    print(f'{(missing/series_data)*100}% of Time Series Data Imputed')

    return time_series

In [12]:
find_missing_data(states)

23226 NG Gas Consumption Data Points
24144 Data Points in Time Series Data
918 Missing Data Points Imputed with 0s
3.802186878727634% of Time Series Data Imputed


,state,plantCode,Longitude,Latitude,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,...,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01
0,MI,57950,-82.619167,43.266389,102105.0,88115.0,81350.0,0.0,0.0,0.0,...,98136.0,0.0,0.0,0.0,0.0,0.0,86083.0,109779.0,99350.0,106351.0
1,MI,57949,-83.447222,43.741111,43145.0,37347.0,42454.0,19712.0,0.0,0.0,...,31588.0,31459.0,10686.0,0.0,0.0,0.0,38845.0,53431.0,43853.0,46659.0
2,MI,54867,-86.644300,46.409400,11883.0,10159.0,10886.0,11237.0,10643.0,11083.0,...,11549.0,10144.0,9584.0,15025.0,20091.0,18337.0,16208.0,15731.0,10704.0,14610.0
3,MI,58427,-84.551667,42.719722,595462.0,512085.0,298345.0,344856.0,463648.0,457361.0,...,547573.0,461526.0,307599.0,456549.0,467034.0,466195.0,253597.0,419370.0,541031.0,597797.0
4,MI,62192,-82.479331,42.773716,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4157779.0,4278830.0,4279821.0,4144307.0,4071685.0,3513872.0,2357711.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,FL,50858,-82.340469,27.954901,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
499,FL,10275,-81.600600,27.911400,49491.0,47142.0,51422.0,51245.0,66844.0,69119.0,...,53962.0,53814.0,65582.0,71830.0,78615.0,78811.0,68981.0,57527.0,53400.0,53814.0
500,FL,7699,-81.849446,27.746369,0.0,0.0,0.0,756735.0,513611.0,1046951.0,...,1054747.0,819635.0,1114587.0,1071691.0,1042457.0,775491.0,846827.0,1115881.0,716660.0,0.0
501,FL,10062,-80.356600,25.835600,29428.0,24034.0,39399.0,33071.0,33795.0,36348.0,...,61371.0,43388.0,33450.0,33277.0,13440.0,18049.0,18112.0,20217.0,12939.0,9498.0
